<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Chapter-8.-Object-References,-Mutability,-and-Recycling" data-toc-modified-id="Chapter-8.-Object-References,-Mutability,-and-Recycling-1">Chapter 8. Object References, Mutability, and Recycling</a></span><ul class="toc-item"><li><span><a href="#Aliasing" data-toc-modified-id="Aliasing-1.1">Aliasing</a></span></li><li><span><a href="#Equality-and-equivalence(identity)" data-toc-modified-id="Equality-and-equivalence(identity)-1.2">Equality and equivalence(identity)</a></span></li><li><span><a href="#Copies-Are-Shallow-by-Default" data-toc-modified-id="Copies-Are-Shallow-by-Default-1.3">Copies Are Shallow by Default</a></span><ul class="toc-item"><li><span><a href="#Cyclic-references" data-toc-modified-id="Cyclic-references-1.3.1">Cyclic references</a></span></li></ul></li><li><span><a href="#Function-parameter-passing" data-toc-modified-id="Function-parameter-passing-1.4">Function parameter passing</a></span></li><li><span><a href="#Mutable-Types-as-Parameter-Defaults:-Bad-Idea" data-toc-modified-id="Mutable-Types-as-Parameter-Defaults:-Bad-Idea-1.5">Mutable Types as Parameter Defaults: Bad Idea</a></span></li><li><span><a href="#Reference-counting-garbage-collection" data-toc-modified-id="Reference-counting-garbage-collection-1.6">Reference counting garbage collection</a></span></li><li><span><a href="#weak-references" data-toc-modified-id="weak-references-1.7">weak references</a></span></li><li><span><a href="#WeakValueDictionary." data-toc-modified-id="WeakValueDictionary.-1.8">WeakValueDictionary.</a></span></li><li><span><a href="#Quirkiness-of-Python-Immutables" data-toc-modified-id="Quirkiness-of-Python-Immutables-1.9">Quirkiness of Python Immutables</a></span><ul class="toc-item"><li><span><a href="#Sharing-of-string-literals-(interning)" data-toc-modified-id="Sharing-of-string-literals-(interning)-1.9.1">Sharing of string literals (interning)</a></span></li></ul></li></ul></li></ul></div>

# Chapter 8. Object References, Mutability, and Recycling

Variables are labels, not boxes

In [247]:
# Variables a and b hold references to the same list,
# not copies of the list

a = [1,2,3]
b = a
a.append(4)
b

[1, 2, 3, 4]

In [248]:
# Variables are assigned to objects only after the objects 
# are created
class Gizmo:
    def __init__(self):
         print('Gizmo id: %d' % id(self))

x = Gizmo()
y = Gizmo() * 10 
# variable y was never created, because the exception 
# happens while the right-hand side of the assignment 
# was being evaluated

Gizmo id: 140230736824592
Gizmo id: 140230736833984


TypeError: unsupported operand type(s) for *: 'Gizmo' and 'int'

In [250]:
print(x)
print(y)

NameError: name 'y' is not defined

## Aliasing

charles and lewis refer to the same object


In [252]:
charles = {'name': 'Charles L. Dodgson', 'born': 1832}
lewis = charles  
lewis is charles

True

In [253]:
id(charles), id(lewis)  

(140230733458560, 140230733458560)

In [254]:
lewis['balance'] = 950  
charles

{'name': 'Charles L. Dodgson', 'born': 1832, 'balance': 950}

alex and charles compare equal, but alex is not an alias for charles

In [255]:
alex = {'name': 'Charles L. Dodgson', 
        'born': 1832, 'balance': 950}
alex == charles

True

In [256]:
# The is operator compares the identity of two objects; 
# the id() function returns an integer representing its 
# identity
alex is charles

False

## Equality and equivalence(identity)

In [257]:
# Example using an unhashable tuple
t1 = (1,2,[3])
t2 = (1,2,[3])
t1 == t2

True

In [258]:
id(t1[-1])

140230733266304

In [259]:
t3 = t1
t3 is t1

True

In [260]:
t1[-1].append(4)
t1

(1, 2, [3, 4])

In [261]:
id(t1[-1])


140230733266304

In [264]:
# equality is broken from element value change
t1 == t2

False

In [270]:
# equivalance (identity) is however not broken
t3 is t1

True

In [271]:
print('t1 -> {}'.format(t1))
print('t2 -> {}'.format(t2))

t1 -> (1, 2, [3, 4])
t2 -> (1, 2, [3])


## Copies Are Shallow by Default
the outermost container is duplicated, but the copy is filled with references to the same items held by the original container.

This saves memory and causes no problems if all the items are immutable. But if there are mutable items, this may lead to unpleasant surprises.

In [273]:
l1 = [3, [66, 55, 44], (7, 8, 9)]
l2 = list(l1)      # l2 is a shallow copy of l1
l1.append(100)     # Appending to l1 has no effect on l2
l1[1].remove(55)   # Changing the l1 mutable element affects l2
print('l1:', l1)
print('l2:', l2)
l2[1] += [33, 22]  # Changing the l2 mutable element affects l1
l2[2] += (10, 11)  # Changing the l2 immutable with += has no effect on l1
print('l1:', l1)
print('l2:', l2)

l1: [3, [66, 44], (7, 8, 9), 100]
l2: [3, [66, 44], (7, 8, 9)]
l1: [3, [66, 44, 33, 22], (7, 8, 9), 100]
l2: [3, [66, 44, 33, 22], (7, 8, 9, 10, 11)]


In [274]:
class Bus:

    def __init__(self, passengers=None):
        if passengers is None:
            self.passengers = []
        else:
            self.passengers = list(passengers)

    def pick(self, name):
        self.passengers.append(name)

    def drop(self, name):
        self.passengers.remove(name)

In [275]:
import copy
bus1 = Bus(['Alice', 'Bill', 'Claire', 'David'])
bus2 = copy.copy(bus1) # bus2 is a shallow copy
bus3 = copy.deepcopy(bus1) # bus3 is a deep copy
id(bus1.passengers), id(bus2.passengers), id(bus3.passengers)

(140230733592128, 140230733592128, 140230733245504)

In [276]:
id(bus1), id(bus2), id(bus3)
bus1.drop('Bill') # affects shallow copy bus2 but not deep copy bus3
bus2.passengers

['Alice', 'Claire', 'David']

In [277]:
bus3.passengers

['Alice', 'Bill', 'Claire', 'David']

### Cyclic references

In [278]:
a = [10, 20] # b refers to a
b = [a, 30]  # and then is appended to a
b

[[10, 20], 30]

In [279]:
a.append(b)
a

[10, 20, [[...], 30]]

In [280]:
c = copy.deepcopy(a) # deepcopy still manages to copy a
c

[10, 20, [[...], 30]]

## Function parameter passing
The only mode of parameter passing in Python is call by sharing.

Call by sharing means that each formal parameter of the function gets a copy of each reference in the arguments. In other words, the parameters inside the function become aliases of the actual arguments.

a function may change any mutable object passed as a parameter, but it cannot change the identity of those objects 

In [281]:
def f(a,b):
    a += b # mutable objects are changed, immutables are re-created
    return a

In [282]:
x, y = 1,2
f(x,y)

3

In [283]:
x,y # immutable x is unchanged

(1, 2)

In [284]:
a = [1,2]
b = [3,4]
f(a,b)

[1, 2, 3, 4]

In [287]:
a,b # mutable a is changed

([1, 2, 3, 4], [3, 4])

In [288]:
t = (10, 20)
u = (30, 40)
f(t,u)

(10, 20, 30, 40)

In [289]:
t,u # immutable t is unchanged

((10, 20), (30, 40))

## Mutable Types as Parameter Defaults: Bad Idea
The problem is that instances that don’t get passed an initial list end up sharing the same list among themselves.

TIP:
Unless a method is explicitly intended to mutate an object received as argument, you should think twice before aliasing the argument object by simply assigning it to an instance variable in your class. If in doubt, make a copy.

In [290]:
class HauntedBus:
    """A bus model haunted by ghost passengers"""

    def __init__(self, passengers=[]):   
        self.passengers = passengers   

    def pick(self, name):
        self.passengers.append(name)   

    def drop(self, name):
        self.passengers.remove(name)

In [291]:
bus1 = HauntedBus(['Alice', 'Bill'])
bus1.passengers

['Alice', 'Bill']

In [292]:
bus1.pick('Charlie')
bus1.drop('Alice')
bus1.passengers

['Bill', 'Charlie']

In [293]:
bus2 = HauntedBus() 
bus2.pick('Carrie') # The default is no longer empty!
bus2.passengers

['Carrie']

In [294]:
bus3 = HauntedBus()
bus3.passengers # Now Dave, picked by bus3, appears in bus2

['Carrie']

In [295]:
bus3.pick('Dave')
bus2.passengers

['Carrie', 'Dave']

In [296]:
# bus2.passengers and bus3.passengers refer to the same list
bus2.passengers is bus3.passengers 

True

In [297]:
HauntedBus.__init__.__defaults__

(['Carrie', 'Dave'],)

In [298]:
class FixedBus:
    """A bus model haunted by ghost passengers"""

    def __init__(self, passengers=None): # passengers can now be 
                                         # any iterable type
        if not passengers:
            self.passengers = []
        else:
            self.passengers = list(passengers) # Make a copy of the iterable

    def pick(self, name):
        self.passengers.append(name)   

    def drop(self, name):
        self.passengers.remove(name)

In [299]:
bus2 = FixedBus()
bus2.pick('Carrie')
bus2.passengers

['Carrie']

In [300]:
bus3 = FixedBus()
bus3.passengers

[]

In [301]:
bus3.pick('Dave')
bus2.passengers

['Carrie']

In [302]:
bus1 = FixedBus({'Alice', 'Bill'}) # ctor now accepts any iterable
bus1.passengers

['Alice', 'Bill']

## Reference counting garbage collection

In CPython 1, each object keeps count of how many references point to it. As soon as that refcount reaches zero, the object is immediately destroyed: The interpreter calls the __del__ method on the object (if defined) and then frees the memory allocated to the object. 

In CPython 2.0, a generational garbage collection algorithm was added to detect groups of objects involved in reference cycles—which may be unreachable even with outstanding references to them, when all the mutual references are contained within the group.

In [303]:
import weakref
s1 = {1, 2, 3}
s2 = s1 

def bye():      
    print('object destroyed...')

ender = weakref.finalize(s1, bye)  # register a callback to be called 
                                   # when an object is destroyed.
ender.alive 

True

In [304]:
del s1    # del does not delete an object, just a reference to it
ender.alive

True

In [305]:
s2 = 'spam' # Rebinding the last reference to s1
            # makes {1, 2, 3} unreachable. 
    
            # s1 is destroyed, the bye callback is invoked, 
            # and ender.alive becomes False
ender.alive

object destroyed...


False

## weak references
Weak references to an object do not increase its reference count and therefore does not prevent the referent from being garbage collected

Weak references are useful in caching applications because you don’t want the cached objects to be kept alive just because they are referenced by the cache.

In [306]:
import weakref
a_set = {0, 1}
wref = weakref.ref(a_set)  
wref

<weakref at 0x7f8a031a2540; to 'set' at 0x7f8a03caa820>

In [307]:
wref()  

{0, 1}

In [308]:
a_set = {2, 3, 4}  
wref()  

{0, 1}

In [309]:
wref() is None  # {0,1} should have been destroyed.
                # This example doesn't work because of some
                # hidden assignment in notebook environment

False

## WeakValueDictionary. 
This is commonly used for caching

If you need to build a class that is aware of every one of its instances, a good solution is to create a class attribute with a WeakSet to hold the references to the instances. 


Otherwise, if a regular set was used, the instances would never be garbage collected, because the class itself would have strong references to them, and classes live as long as the Python process unless you deliberately delete them

In [315]:
class Cheese:

    def __init__(self, kind):
        self.kind = kind

    def __repr__(self):
        return 'Cheese(%r)' % self.kind

In [316]:
import weakref
stock = weakref.WeakValueDictionary()  
catalog = [Cheese('Red Leicester'), Cheese('Tilsit'),
        Cheese('Brie'), Cheese('Parmesan')]
for cheese in catalog:
     stock[cheese.kind] = cheese  


sorted(stock.keys())

['Brie', 'Parmesan', 'Red Leicester', 'Tilsit']

In [317]:
del catalog
sorted(stock.keys()) # last key not destroyed because cheese is global 
                     # and is not cleaned up after loop finishes

['Parmesan']

In [318]:
del cheese # explicitly deleting cheese removes last reference 
           # in weak value dict 
sorted(stock.keys())

[]

## Quirkiness of Python Immutables
they save memory and make the interpreter faster

In [319]:
# immutables don't make copies, but return references

In [320]:
# A tuple built from another is actually the same exact tuple
# The same behavior can be observed with instances of str, bytes, 
# and frozenset
t1 = (1, 2, 3)
t2 = tuple(t1)
t2 is t1

True

In [321]:
t3 = t1[:]
t3 is t1 

True

In [322]:
import copy
t4 = copy.copy(t1)
t4 is t1

True

### Sharing of string literals (interning)

In [323]:
t1 = (1, 2, 3)
t3 = (1, 2, 3)
t3 is t1

False

In [324]:
s1 = 'ABC'
s2 = 'ABC'
s1 is s2

True